In [4]:
# Import Dependencies
import os
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from splinter import Browser



def scrape():
    
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)
    
    #Visit Mars News url  using splinter module

    Mars_url = 'https://redplanetscience.com/'
    browser.visit(Mars_url)
    #create HTMl Object
    html = browser.html
    #parse HTML with beautiful soup
    Mars_soup = bs(html, 'html.parser')

    # Extract title text
    mars_news_title = Mars_soup.find('div', class_= 'content_title').text
    print(mars_news_title)
    # Extract Paragraph text
    mars_news_paragraph=Mars_soup.find('div',class_='article_teaser_body').text
    
    
    Mars_url2 = ('https://spaceimages-mars.com/')

    response = requests.get(Mars_url2)
    soup = BeautifulSoup(response.text, 'html.parser')



    #pull images from website
    images = soup.findAll('img')
    
    
    # pull image link
    pic_src = []
    for image in images:
        pic = image['src']
        pic_src.append(pic)

    featured_image_url = 'https://spaceimages-mars.com/' + pic
    
    
    
    # Visit the Mars facts webpage and scrape table data into Pandas
    Mars_url3 = "http://space-facts.com/mars/"
    browser.visit(Mars_url3)
    
    
    # place data into a dataframe, clean it up and output it into an HTML table

    grab=pd.read_html(Mars_url3)
    mars_data=pd.DataFrame(grab[0])
    mars_data.columns=['Mars','Data']
    mars_table=mars_data.set_index("Mars")
    marsdata = mars_table.to_html(classes='marsdata')
    marsdata=marsdata.replace('\n', ' ')
    
    
    # Visit the USGS Astrogeology site
    USGS_url="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
    short_url="https://astrogeology.usgs.gov"
    
    
    browser.visit(USGS_url)
    html = browser.html
    soup = bs(html, 'html.parser')
    main_url = soup.find_all('div', class_='item')
    titles=[]
    hemisphere_img_urls=[]
    
    for x in main_url:
        title = x.find('h3').text
        url = x.find('a')['href']
        hem_img_url= short_url+url
        #print(hem_img_url)
        browser.visit(hem_img_url)
        html = browser.html
        soup = bs(html, 'html.parser')
        hemisphere_img_original= soup.find('div',class_='downloads')
        hemisphere_img_url=hemisphere_img_original.find('a')['href']
    
    
    img_data=dict({'title':title, 'img_url':hemisphere_img_url})
    hemisphere_img_urls.append(img_data)
    
    
    data = {"News Header": mars_news_title, "News Article": mars_news_paragraph, "Mars Image": featured_image_url, "Mars Facts": marsdata, "Hemispheres": hemisphere_img_urls}
    return data